## Clasificación de textos utilizando Transformers

La clasificación de textos consiste en, dado un texto, asignarle una entre varias categorías. Algunos ejemplos de esta tarea son:

- dado un tweet, categorizar su connotación como positiva, negativa o neutra.
- dado un post de Facebook, clasificarlo como portador de un lenguaje ofensivo o no.  

En la actividad exploraremos cómo utilizar soluciones *out of the box* para esta tarea incluidas en la librería [Transformers](https://huggingface.co/transformers/) y su aplicación en un caso de uso típico: realizar un análisis básico del estado de opinión sobre un producto turístico a partir de los comentarios de los usuarios.



**Instrucciones:**

- siga las indicaciones y comentarios en cada apartado.


**Después de esta actividad nos habremos familiarizado con:**
- seleccionar e instanciar modelos pre-entrenados para realizar clasificación de textos.
- cómo instanciar un *pipeline* para la clasificación de textos utilizando la librería Transformers.
- utilizar este *pipeline* para clasificar nuevos textos.

**Requerimientos**
- python 3.6.12 - 3.8
- tensorflow==2.3.0
- transformers==4.1.0
- pandas==1.1.5
- plotly==4.13.0


## Instalación de librerías e importación de dependencias.

Para comenzar, es preciso instalar las dependencias y realizar los imports necesarios.

Ejecute las siguientes casillas prestando atención a las instrucciones adicionales en los comentarios.

In [ ]:
# instalar librerías. Esta casilla es últil por ejemplo si se ejecuta el cuaderno en Google Colab
# Note que existen otras dependencias como tensorflow==2.3.0, etc. que en este caso se encontrarían ya instaladas
%%capture
!pip install transformers==4.1.0 pandas==1.1.5 plotly==4.13.0 

print('Done!')

In [ ]:
from transformers import TextClassificationPipeline, DistilBertTokenizer, TFDistilBertForSequenceClassification, ModelCard
import pandas as pd
import plotly.graph_objects as go
from collections import Counter

print('Done!')

La librería Transformers provee diferentes modelos listos para usar en la tarea de clasificación de textos. Una forma flexible de lograrlo consiste en:

- seleccionar un modelo pre-entrenado adecuado para la tarea. Podemos examinar los modelos disponibles en [https://huggingface.co/models](https://huggingface.co/models). Estaremos utilizando el llamado [distilbert-base-uncased-finetuned-sst-2-english](https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english) que permite clasificar un texto en idioma inglés de acuerdo con su connotación **positiva** o **negativa**.

- instanciar el modelo y su correspondiente tokenizador.

- crear un pipeline para la clasificación de textos, en este caso utilizando la clase [TextClassificationPipeline](https://huggingface.co/transformers/main_classes/pipelines.html#transformers.TextClassificationPipeline).

- utilizar el pipeline para clasificar textos.


Ejecute la siguiente celda para instanciar el modelo y el correspondiente tokenizador.

**Note que:**
- la práctica recomendada al crear un nuevo modelo para Transformers es hacerlo disponible mediante un fichero que contiene los elementos necesarios para su posterior uso, como son el modelo, el tokenizador y una tarjeta con metadatos sobre el modelo. 

- es conveniente indagar sobre el modelo base utilizado, en este caso **DistilBert**, esto permitirá seleccionar las clases adecuadas para instanciar el modelo.


In [ ]:
trained_model_name = 'distilbert-base-uncased-finetuned-sst-2-english'

tokenizer = DistilBertTokenizer.from_pretrained(trained_model_name)
model = TFDistilBertForSequenceClassification.from_pretrained(trained_model_name)
modelcard = ModelCard.from_pretrained(trained_model_name)

print('Done!')

Una vez instanciado el modelo y el tokenizador, instanciamos el pipeline de clasificación de textos. 

Ejecute la siguiente celda para crear una instancia de TextClassificationPipeline a partir del modelo y tokenizador.

In [ ]:
classifier = TextClassificationPipeline(model=model, tokenizer=tokenizer, modelcard=None, framework='tf', task='sentiment-analysis', return_all_scores=False)

print('Done!')

Ejecute la siguiente celda para clasificar una la frase. Alternativamente, puede modificar el texto incluyendo uno de su preferencia. Recuerde que debe ser en idioma inglés.

In [ ]:
text = "Natural Language Processing is a very interesting subject."
output = classifier(text)
print(output)

print('Done!')

### Análisis de opiniones

Finalmente, utilizaremos nuestro pipeline para realizar un análisis básico del estado de opinión en una colección de tweets sobre *vacunas* (*). Para esto es necesario:

- leer los datos, en este caso, disponibles en un archivo ".csv" y obtener una lista con cada texto.
- utilizar el pipeline para clasificar los textos
- construir un gráfico de barras mostrando las cantidades de opiniones positivas y negativas.

Ejecute la siguiente celda para leer los datos. Existen diferentes opciones, entre estas:

- montar nuestra partición de Google Drive y leer un fichero desde esta.

- leer los datos desde un fichero en una carpeta local.

- leer los datos directamente de un URL.

(*) Los datos son un subconjunto de los provistos en [SemEval 2017, Tarea 4, D](http://alt.qcri.org/semeval2017/task4/)

In [ ]:
# descomente las siguientes 3 líneas para leer datos desde Google Drive,sumiendo que se trata de un fichero llamado review.csv localizado dentro de una carpeta llamada 'Datos' en su Google Drive.
#from google.colab import drive
#drive.mount('/content/drive')
#path = '/content/drive/MyDrive/Datos/sentiment-analysis-review.csv'

# descomente la siguiente línea para leer los datos desde un archivo local, por ejemplo, asumiendo que se encuentra dentro de un directorio llamado sample_data
path = './sample_data/sentiment-analysis-review.csv'

# descomente la siguiente línea para leer datos desde un URL
#path = 'https://github.com/TeachingTextMining/TextClassification/raw/main/02-SA-Transformers-Basic/sample_data/sentiment-analysis-review.csv'


# leer los datos
corpus = pd.read_csv(path, sep=',')
texts = corpus['Tweet'].to_list()
print(texts[0])

print('Done!')

Ejecute la siguiente celda para clasificar los textos. Tenga en cuenta que, en dependencia del entorno de ejecución, la cantidad de textos y su longitud, la ejecución puede tardar varios minutos o requerir una cantidad de memoria no disponible.

In [ ]:
opinions = classifier(texts)
print(opinions[0])

Ejecute la siguiente celda para obtener algunas estadísticas básicas sobre los resultados.

In [ ]:
pred_opinions = [opinion['label'] for opinion in opinions]

# obtener algunas estadísticas sobre la predicción en el conjunto de pruebas
categories = ['POSITIVE', 'NEGATIVE']
hist = Counter(pred_opinions) 

colors = ['red', 'orange', 'yellow', 'lightgreen', 'darkgreen']
fig = go.Figure(layout=go.Layout(height=400, width=600))
fig.add_trace(go.Bar(x=categories, y=[hist[cat] for cat in sorted(hist.keys())], marker_color=colors))
fig.show()

print('Done!')